In [ ]:
import pandas as pd
import numpy as np
import warnings
import os

# Suppress warnings from division by zero, etc.
warnings.filterwarnings('ignore')

# --- 0. CRITICAL HELPER FUNCTION ---
# This function standardizes district names across all files.
def clean_district_name(district_name):
    if pd.isna(district_name):
        return None

    # Convert to string, make uppercase, remove whitespace
    name = str(district_name).upper().strip()

    return name

# --- 1. DEFINE YOUR FILE PATHS ---
print("--- Phase 1: Data Engineering Started (Predictive Model) ---")

# --- 2023-24 Paths (for Features) ---
path_profile_1_23_24 = "/home/UDISE 2023-24/100_prof1.csv"
path_enrolment_23_24 = "/home/UDISE 2023-24/100_enr1.csv"
path_teachers_23_24 = "/home/UDISE 2023-24/100_tch.csv"
path_facility_23_24 = "/home/UDISE 2023-24/100_fac.csv"

# --- 2024-25 Path (for Outcome) ---
path_enrolment_24_25 = "/home/UDISE 2024-25/100_enr1.csv"

# --- NFHS Path (for Contextual Features) ---
path_nfhs = '/content/datafile.csv'


--- Phase 1: Data Engineering Started (Predictive Model) ---


In [ ]:
# --- 2. CREATE TARGET VARIABLES (Y) ---
print("Step 1/4: Calculating District Transition/Dropout Rates (Sec and HSec)...")

try:
    # --- Load Master School Profile (from 23-24) ---
    profiles = pd.read_csv(path_profile_1_23_24)
    # Keep only the pseudocode and district, and clean the district name
    profiles_dist = profiles[['pseudocode', 'district']].drop_duplicates()
    profiles_dist['district'] = profiles_dist['district'].apply(clean_district_name)

    # --- Process 2023-24 Enrolment (C8 and C10) ---
    enrol_23_24 = pd.read_csv(path_enrolment_23_24)
    # Sum C8 boys and girls for each row
    enrol_23_24['total_c8_row'] = enrol_23_24['c8_b'] + enrol_23_24['c8_g']
    # Sum C10 boys and girls for each row
    enrol_23_24['total_c10_row'] = enrol_23_24['c10_b'] + enrol_23_24['c10_g']

    # Sum all rows to get total enrolment for each school
    school_23 = enrol_23_24.groupby('pseudocode').agg(
        total_c8_2023=('total_c8_row', 'sum'),
        total_c10_2023=('total_c10_row', 'sum')
    ).reset_index()

    # --- Process 2024-25 Enrolment (C9 and C11) ---
    enrol_24_25 = pd.read_csv(path_enrolment_24_25)
    # Sum C9 boys and girls for each row
    enrol_24_25['total_c9_row'] = enrol_24_25['c9_b'] + enrol_24_25['c9_g']
    # Sum C11 boys and girls for each row
    enrol_24_25['total_c11_row'] = enrol_24_25['c11_b'] + enrol_24_25['c11_g']

    # Sum all rows to get total enrolment for each school
    school_24 = enrol_24_25.groupby('pseudocode').agg(
        total_c9_2024=('total_c9_row', 'sum'),
        total_c11_2024=('total_c11_row', 'sum')
    ).reset_index()

    # --- Merge 2023 and 2024 data at school level ---
    school_transitions = pd.merge(school_23, school_24, on='pseudocode', how='inner')

    # --- Map to District (using 23-24 profile) ---
    school_transitions_dist = pd.merge(school_transitions, profiles_dist, on='pseudocode', how='left')

    # --- Aggregate by District ---
    # Sum all students in each district
    district_transitions = school_transitions_dist.groupby('district').agg(
        total_c8_2023=('total_c8_2023', 'sum'),
        total_c10_2023=('total_c10_2023', 'sum'),
        total_c9_2024=('total_c9_2024', 'sum'),
        total_c11_2024=('total_c11_2024', 'sum')
    ).reset_index()

    # --- Calculate Target Variables (Y) ---
    # Secondary Transition
    district_transitions['Transition_Rate_Sec'] = district_transitions['total_c9_2024'] / district_transitions['total_c8_2023']
    district_transitions['Dropout_Rate_Sec'] = 1 - district_transitions['Transition_Rate_Sec']

    # Higher Secondary Transition
    district_transitions['Transition_Rate_HSec'] = district_transitions['total_c11_2024'] / district_transitions['total_c10_2023']
    district_transitions['Dropout_Rate_HSec'] = 1 - district_transitions['Transition_Rate_HSec']

    # Handle division by zero
    district_transitions.replace([np.inf, -np.inf], np.nan, inplace=True)

    district_targets_df = district_transitions[['district', 'Dropout_Rate_Sec', 'Dropout_Rate_HSec']]
    print("  > Targets 'Dropout_Rate_Sec' and 'Dropout_Rate_HSec' created.")

except FileNotFoundError as e:
    print(f"Error: File not found. {e}")
    print("Please make sure your files are in 'UDISE 2023-24' and 'UDISE 2024-25' folders.")
    exit()
except KeyError as e:
    print(f"Error: Missing column {e}. Check your CSV headers in enrolment/profile files.")
    exit()


Step 1/4: Calculating District Transition/Dropout Rates (Sec and HSec)...
  > Targets 'Dropout_Rate_Sec' and 'Dropout_Rate_HSec' created.


In [ ]:

# --- 3. CREATE SCHOOL-BASED FEATURES (X1) ---
print("Step 2/4: Creating School-Based Features (PTR, Facilities) from 2023-24 data...")
try:
    # --- 3a. Calculate Average PTR (using 2023-24 data) ---
    teachers_23_24 = pd.read_csv(path_teachers_23_24)

    # Get all enrolment columns from the 2023-24 file
    enrol_23_24_for_ptr = pd.read_csv(path_enrolment_23_24)
    enrol_cols = [col for col in enrol_23_24_for_ptr.columns if col.startswith('c') and (col.endswith('_b') or col.endswith('_g'))]
    enrol_23_24_for_ptr['total_enrol_row'] = enrol_23_24_for_ptr[enrol_cols].sum(axis=1)

    # Sum all rows to get total enrolment for each school
    school_enrol_23 = enrol_23_24_for_ptr.groupby('pseudocode')['total_enrol_row'].sum().reset_index(name='total_enrolment')

    # Merge with teachers data
    school_ptr_data = pd.merge(school_enrol_23, teachers_23_24[['pseudocode', 'total_tch']], on='pseudocode', how='left')

    # Calculate PTR per school
    school_ptr_data['PTR_school'] = school_ptr_data['total_enrolment'] / school_ptr_data['total_tch']
    school_ptr_data.replace([np.inf, -np.inf], np.nan, inplace=True) # Handle schools with 0 teachers

    # Map to district (using 23-24 profile)
    school_ptr_dist = pd.merge(school_ptr_data, profiles_dist, on='pseudocode', how='left')

    # Aggregate by district (taking the mean PTR)
    district_ptr = school_ptr_dist.groupby('district')['PTR_school'].mean().reset_index(name='Avg_PTR_district_23_24')

    # --- 3b. Calculate Facility Scores (using 2023-24 data) ---
    facility_23_24 = pd.read_csv(path_facility_23_24)

    # Create binary feature columns (based on schema: 1 = Yes)
    facility_23_24['has_func_toilets'] = ((facility_23_24['total_boys_func_toilet'] > 0) & (facility_23_24['total_girls_func_toilet'] > 0)).astype(int)
    facility_23_24['has_library'] = (facility_23_24['library_availability'] == 1).astype(int)
    facility_23_24['has_electricity'] = (facility_23_24['electricity_availability'] == 1).astype(int) # 1=Yes, 2=No, 3=Yes but not functional

    # Map to district (using 23-24 profile)
    facility_dist = pd.merge(facility_23_24[['pseudocode', 'has_func_toilets', 'has_library', 'has_electricity']], profiles_dist, on='pseudocode', how='left')

    # Aggregate by district (mean of 0/1 columns = percentage)
    district_facilities = facility_dist.groupby('district')[['has_func_toilets', 'has_library', 'has_electricity']].mean().reset_index()

    # Rename for clarity
    district_facilities.rename(columns={
        'has_func_toilets': '%_Schools_Func_Toilets_23_24',
        'has_library': '%_Schools_Library_23_24',
        'has_electricity': '%_Schools_Electricity_23_24'
    }, inplace=True)

    # --- 3c. Combine School Features (X1) ---
    district_features_school_df = pd.merge(district_ptr, district_facilities, on='district', how='outer')
    print("  > School-based features (X1) created from 2023-24 data.")

except FileNotFoundError as e:
    print(f"Error: File not found. {e}")
    exit()
except KeyError as e:
    print(f"Error: Missing column {e}. Check your CSV headers in teacher/facility files.")
    exit()



Step 2/4: Creating School-Based Features (PTR, Facilities) from 2023-24 data...
  > School-based features (X1) created from 2023-24 data.


In [ ]:
# --- 4. LOAD & CLEAN NFHS FEATURES (X2) ---
print("Step 3/4: Loading and Cleaning NFHS-5 Features (X2)...")
try:
    nfhs_df = pd.read_csv(path_nfhs)

    # --- 1. Define the exact NFHS District Column ---
    NFHS_DISTRICT_COL = 'District Names' # From your provided list

    # --- 2. Define the exact NFHS Feature Columns we need ---
    NFHS_FEATURES_COLS = [
        # For "Parental Education"
        'Women (age 15-49)  with 10 or more years of schooling (%)',
        'Women (age 15-49) who are literate4 (%)',
        'Female population age 6 years and above who ever attended school (%)',

        # For "Household Income" (Wealth Proxies)
        'Population living in households with an improved drinking-water source1 (%)',
        'Population living in households that use an improved sanitation facility2 (%)',
        'Population living in households with electricity (%)',

        # For "Employment Status" (Proxy)
        'Households with any usual member covered under a health insurance/financing scheme (%)',
        'Women age 20-24 years married before age 18 years (%)',
        'Women age 15-19 years who were already mothers or pregnant at the time of the survey (%)',
        'Current Use of Family Planning Methods (Currently Married Women Age 15-49  years) - Any method6 (%)',

    ]

    # --- 3. Define short, clean names for our final dataset ---
    rename_map = {
        # District Column
        'District Names': 'district',

        # --- For "Parental Education" ---
        'Women (age 15-49)  with 10 or more years of schooling (%)': 'NFHS_Women_10_plus_School',
        'Women (age 15-49) who are literate4 (%)': 'NFHS_Women_Literate',
        'Female population age 6 years and above who ever attended school (%)': 'NFHS_Female_Ever_Attended_School',

        # --- For "Household Income" (Wealth Proxies) ---
        'Population living in households with an improved drinking-water source1 (%)': 'NFHS_Improved_Water',
        'Population living in households that use an improved sanitation facility2 (%)': 'NFHS_Improved_Sanitation',
        'Population living in households with electricity (%)': 'NFHS_Electricity',

        # --- For "Employment Status" & Social Proxies ---
        'Households with any usual member covered under a health insurance/financing scheme (%)': 'NFHS_Health_Insurance',
        'Women age 20-24 years married before age 18 years (%)': 'NFHS_Early_Marriage',
        'Women age 15-19 years who were already mothers or pregnant at the time of the survey (%)': 'NFHS_Teen_Pregnancy',
        'Current Use of Family Planning Methods (Currently Married Women Age 15-49  years) - Any method6 (%)': 'NFHS_Family_Planning_Modern'
    }
    # --- 4. Check that all our needed columns exist ---
    if NFHS_DISTRICT_COL not in nfhs_df.columns:
        print(f"Error: NFHS district column '{NFHS_DISTRICT_COL}' not found in {path_nfhs}")
        print(f"Available columns: {nfhs_df.columns.to_list()}")
        exit()

    valid_cols = []
    for col in NFHS_FEATURES_COLS:
        if col not in nfhs_df.columns:
            print(f"Warning: NFHS feature column '{col}' not found. Will be skipped.")
        else:
            valid_cols.append(col)

    # --- 5. Select, Clean, and Rename ---

    # Select only the columns we need
    nfhs_features_df = nfhs_df[[NFHS_DISTRICT_COL] + valid_cols].copy()

    # Rename columns to our short, clean names
    nfhs_features_df.rename(columns=rename_map, inplace=True)

    # Clean the district names using the same function
    nfhs_features_df['district'] = nfhs_features_df['district'].apply(clean_district_name)

    # Remove any rows with missing data
    nfhs_features_df.dropna(inplace=True)

    print(f"  > NFHS data loaded. Found {len(valid_cols)} relevant features.")

except FileNotFoundError:
    print(f"Error: Could not find NFHS file at {path_nfhs}.")
    exit()
except KeyError as e:
    print(f"Error: A column was not found. {e}")
    exit()


Step 3/4: Loading and Cleaning NFHS-5 Features (X2)...
  > NFHS data loaded. Found 10 relevant features.


In [ ]:

# --- 5. MERGE ALL DATAFRAMES ---
print("Step 4/4: Merging all dataframes...")

# 1. Merge Y (targets) and X1 (school features)
merged_school_df = pd.merge(district_targets_df, district_features_school_df, on='district', how='inner')

# 2. Merge with X2 (NFHS features)
# Use 'inner' join to keep only districts that exist in ALL datasets
final_df = pd.merge(merged_school_df, nfhs_features_df, on='district', how='inner')

# Drop any rows that have missing values from all the merging
final_df.dropna(inplace=True)

print("\n--- Phase 1 Complete! ---")
print(f"Final analysis-ready dataset has {final_df.shape[0]} rows (districts) and {final_df.shape[1]} columns.")
print("\nFinal DataFrame Head:")
print(final_df.head())

# Optional: Save the final dataset to a new CSV
final_df.to_csv('analysis_ready_dataset.csv', index=False)
print("\nFinal dataset saved to 'analysis_ready_dataset.csv'")

Step 4/4: Merging all dataframes...

--- Phase 1 Complete! ---
Final analysis-ready dataset has 597 rows (districts) and 17 columns.

Final DataFrame Head:
     district  Dropout_Rate_Sec  Dropout_Rate_HSec  Avg_PTR_district_23_24  \
0    ADILABAD          0.062568           0.361965               36.764144   
1  AGAR MALWA          0.153489           0.393978               37.800578   
2        AGRA          0.143208           0.234261               28.788924   
3  AHMADNAGAR         -0.000984           0.225598               26.103684   
4      AIZAWL          0.038403           0.190961               31.991118   

   %_Schools_Func_Toilets_23_24  %_Schools_Library_23_24  \
0                      0.446635                 0.815092   
1                      0.676166                 0.929188   
2                      0.934308                 0.827986   
3                      0.877176                 0.877744   
4                      0.918644                 0.430508   

   %_Schools_E